In [270]:
import numpy as np
import pandas as pd

In [271]:
df = pd.read_csv('D:/DataSets for ML/finance.csv')

In [272]:
df.head()

,Sentence,Sentiment
0,The GeoSolutions technology will leverage Bene...,positive
1,"$ESI on lows, down $1.50 to $2.50 BK a real po...",negative
2,"For the last quarter of 2010 , Componenta 's n...",positive
3,According to the Finnish-Russian Chamber of Co...,neutral
4,The Swedish buyout firm has sold its remaining...,neutral


In [273]:
df.shape

(5842, 2)

In [274]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import contractions

In [275]:
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\vinay\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\vinay\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\vinay\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [276]:
def preprocess(text):
    # 1. Fix contractions such as (don't -> do not)
    text = contractions.fix(text)

    # 2. Lowecase
    text = text.lower()

    # 3. Remove numbers
    text = re.sub(r'\d+', '', text)

    # 4. Remove non-ASCII  characters
    text = re.sub(r'[^\x00-\x7F]+', ' ', text)

    # 5. Remove punctuation
    text = re.sub(r'[^\w\s]', '', text)
    
    # 6. Remove extra whitespace
    text = re.sub(r'\s+', ' ', text).strip()
    
    # 7. Tokenize
    words = text.split()

    # 8. Remove stopwords and non-alphabetic words
    stop_words = set(stopwords.words('english'))
    words = [word for word in words if word.isalpha() and word not in stop_words]

    # 9. Lemmanization
    lemmatizer = WordNetLemmatizer()
    words = [lemmatizer.lemmatize(word) for word in words]

    return ' '.join(words)


In [277]:
df['Sentence'] = df['Sentence'].apply(preprocess)

In [278]:
df.head()

,Sentence,Sentiment
0,geosolutions technology leverage benefon gps s...,positive
1,esi low bk real possibility,negative
2,last quarter componenta net sale doubled eurm ...,positive
3,according finnishrussian chamber commerce majo...,neutral
4,swedish buyout firm sold remaining percent sta...,neutral


In [279]:
df['Sentence'][485]

'split company would series share series k share'

In [280]:
x = df['Sentence']
x

0       geosolutions technology leverage benefon gps s...
1                             esi low bk real possibility
2       last quarter componenta net sale doubled eurm ...
3       according finnishrussian chamber commerce majo...
4       swedish buyout firm sold remaining percent sta...
                              ...                        
5837    rising cost forced packaging producer huhtamak...
5838    nordic walking first used summer training meth...
5839    according shipping company viking line eu deci...
5840    building home improvement trade sale decreased...
5841    helsinki afx kci konecranes said order four ho...
Name: Sentence, Length: 5842, dtype: object

In [281]:
y = df['Sentiment']
y

0       positive
1       negative
2       positive
3        neutral
4        neutral
          ...   
5837    negative
5838     neutral
5839     neutral
5840     neutral
5841    positive
Name: Sentiment, Length: 5842, dtype: object

In [282]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression

In [283]:
vectorizer = CountVectorizer()
x_vect = vectorizer.fit_transform(x)

In [284]:
print(x_vect)

<Compressed Sparse Row sparse matrix of dtype 'int64'
	with 61088 stored elements and shape (5842, 9740)>
  Coords	Values
  (0, 3335)	1
  (0, 8631)	2
  (0, 4967)	1
  (0, 781)	1
  (0, 3419)	1
  (0, 8082)	1
  (0, 6833)	1
  (0, 5073)	2
  (0, 698)	1
  (0, 7693)	1
  (0, 1547)	1
  (0, 6518)	1
  (0, 7173)	1
  (0, 5678)	1
  (0, 1702)	1
  (0, 5795)	1
  (0, 6635)	1
  (0, 1528)	1
  (0, 5571)	1
  (1, 2661)	1
  (1, 5135)	1
  (1, 864)	1
  (1, 7032)	1
  (1, 6610)	1
  (2, 4874)	1
  :	:
  (5840, 7554)	1
  (5840, 2701)	1
  (5840, 1087)	1
  (5840, 5560)	1
  (5840, 8893)	1
  (5840, 2004)	1
  (5840, 3735)	1
  (5840, 4179)	1
  (5841, 3653)	1
  (5841, 5154)	1
  (5841, 3163)	1
  (5841, 7541)	1
  (5841, 8262)	2
  (5841, 808)	1
  (5841, 6118)	1
  (5841, 177)	1
  (5841, 1831)	1
  (5841, 3769)	1
  (5841, 5444)	1
  (5841, 4765)	1
  (5841, 6754)	1
  (5841, 4225)	1
  (5841, 4667)	1
  (5841, 4834)	1
  (5841, 8347)	1


In [285]:
x_train, x_test, y_train, y_test = train_test_split(x_vect, y, test_size=0.2, random_state=42)

In [286]:
model = LogisticRegression()
model.fit(x_train, y_train)

LogisticRegression()

In [287]:
y_pred = model.predict(x_test)
y_pred

array(['neutral', 'positive', 'negative', ..., 'neutral', 'neutral',
       'neutral'], dtype=object)

In [288]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

    negative       0.32      0.22      0.26       175
     neutral       0.72      0.80      0.76       622
    positive       0.76      0.72      0.74       372

    accuracy                           0.69      1169
   macro avg       0.60      0.58      0.59      1169
weighted avg       0.67      0.69      0.68      1169



In [289]:
accuracy_score(y_test, y_pred)

0.6894781864841745

In [290]:
news = "After the split , the company would have 26,885,540 Series A shares and 9,540,000 Series K shares ."
cleaned_news = preprocess(news)
news_vect = vectorizer.transform([cleaned_news])
prediction = model.predict(news_vect)

print(prediction)

['neutral']
